In [129]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [130]:
train_data = pd.read_csv('Train_set.csv')
test_data = pd.read_csv('Test_set.csv')
print("Train Shape:", train_data.shape)
print("Test Shape:", test_data.shape)

Train Shape: (93174, 23)
Test Shape: (39933, 22)


In [131]:
# print(test_data.isnull().sum())
# print(train_data.isnull().sum())

In [132]:



def clean_experience(x):
    if pd.isnull(x):
        return np.nan
    
    x = str(x).lower().strip()
    
    if "<" in x:
        return 2.5   # less than 5 years
    elif "6-10" in x:
        return 8     # between 6 and 10 years
    elif "10+" in x:
        return 10
    else:
        # extract digits if possible
        import re
        nums = re.findall(r'\d+', x)
        if len(nums) == 1:
            return float(nums[0])
        elif len(nums) == 2:
            # take midpoint of range
            return (float(nums[0]) + float(nums[1])) / 2
        else:
            return np.nan

# Apply to your DataFrame
train_data['job_experience'] = train_data['job_experience'].apply(clean_experience)
test_data['job_experience'] = test_data['job_experience'].apply(clean_experience)
# Handle missing values (simple example)
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
test_data.fillna(test_data.median(numeric_only=True), inplace=True)

In [133]:
# Encode categorical variables
label_enc = LabelEncoder()
for col in train_data.select_dtypes(include=['object']).columns:
    train_data[col] = label_enc.fit_transform(train_data[col].astype(str))
    if col in test_data.columns:
        test_data[col] = label_enc.transform(test_data[col].astype(str))

In [134]:
X = train_data.drop(['default', 'ID'], axis=1, errors='ignore')
y = train_data['default']


# 4️⃣ Define columns
ordinal_cols = ['loan_grade', 'loan_subgrade']
categorical_cols = [
    'home_ownership', 'income_verification_status',
    'loan_purpose', 'application_type', 'state_code'
]
numeric_cols = [
    'loan_amnt', 'loan_term', 'interest_rate', 'job_experience', 
    'annual_income', 'debt_to_income', 'delinq_2yrs', 'public_records',
    'revolving_balance', 'total_acc', 'interest_receive',
    'last_week_pay', 'total_current_balance', 'total_revolving_limit'
]

In [135]:

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
ordinal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('ord', ordinal_transformer, ordinal_cols),
    ('cat', categorical_transformer, categorical_cols)
])

In [ ]:
# 6️⃣ Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 7️⃣ Logistic Regression Model
# log_model = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', LogisticRegression(max_iter=1000))
# ])

# log_model.fit(X_train, y_train)
# y_pred_log = log_model.predict(X_test)
# print("🔹 Logistic Regression Results")
# print("Accuracy:", accuracy_score(y_test, y_pred_log))
# print("ROC AUC:", roc_auc_score(y_test, y_pred_log))
# print(confusion_matrix(y_test, y_pred_log))
# print(classification_report(y_test, y_pred_log))

# 8️⃣ Random Forest Model
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("\n🔹 Random Forest Results")
# print("Accuracy:", accuracy_score(y_test, y_pred_rf))
# print("ROC AUC:", roc_auc_score(y_test, y_pred_rf))
# print(confusion_matrix(y_test, y_pred_rf))
# print(classification_report(y_test, y_pred_rf))